In [1]:
import piano_reduction.tools as pr
import piano_reduction.compute_features as cf
from piano_reduction.classes import ScoreData, Reduction
from sklearn import metrics

Using TensorFlow backend.


In [2]:
reductions = {}
for i in [2, 6, 8]:
    reductions['%d GRU_1' % i] = Reduction.load('models/20190317b/GRU_1 %d.pkl' % i)
    reductions['%d GRU_2' % i] = Reduction.load('models/20190317b/GRU_2 %d.pkl' % i)
    reductions['%d GRU_1 -chord' % i] = Reduction.load('models/20190317b/GRU_1 without chord %d.pkl' % i)
    reductions['%d GRU_2 -chord' % i] = Reduction.load('models/20190317b/GRU_2 without chord %d.pkl' % i)
    try:
        reductions['%d GRU_1 -new' % i] = Reduction.load('models/20190318b/GRU_1 %d.pkl' % i)
    except:
        pass
    try:
        reductions['%d GRU_2 -new' % i] = Reduction.load('models/20190318b/GRU_2 %d.pkl' % i)
    except:
        pass

In [11]:
reductions = {}
for i in [8]:
    for j in [1, 2, 3, 4, 8, 12, 16, 20]:
        reductions['%d (%d) GRU_1' % (i, j)] = Reduction.load('models/20190318l/GRU_1 (%d) %d.pkl' % (j, i))
        reductions['%d (%d) GRU_2' % (i, j)] = Reduction.load('models/20190318l/GRU_2 (%d) %d.pkl' % (j, i))

In [12]:
reductions.keys()

dict_keys(['8 (1) GRU_1', '8 (1) GRU_2', '8 (2) GRU_1', '8 (2) GRU_2', '8 (3) GRU_1', '8 (3) GRU_2', '8 (4) GRU_1', '8 (4) GRU_2', '8 (8) GRU_1', '8 (8) GRU_2', '8 (12) GRU_1', '8 (12) GRU_2', '8 (16) GRU_1', '8 (16) GRU_2', '8 (20) GRU_1', '8 (20) GRU_2'])

In [4]:
for i in reductions:
    if 'GRU_1' in i:
        reductions[i].params['length'] = 20

In [27]:
for i in [2, 6, 8]:
    dfs = pr.load_cosi(i, use_cache=False)
    score = ScoreData(dfs)
    score.save('score_data/cosi_%d.pkl' % i)

Creating df/cosi/00correct.musicxml
P1: Oboes I, II: Oboe 0
P2: Clarinetto I, II in Do/C: C Clarinet 0
P3: Fagotto I, II: Bassoon 0
P4: Clarino I, II in Do/C: C Clarinet 0
P5: Timpani in Do-Sol/C-G: Timpani 0
P6: Violino I: Violins 0
P7: Violino II: Violins 0
P8: Viola I, II: Violas 0
P9: Violoncello e Basso: Violoncellos 0
Creating df/cosi/02.xml
P1: Piano: Piano 0
P1: Piano: Piano 0
Creating df/merged_cosi/2.pkl
Creating df/cosi/00correct.musicxml
P1: Oboes I, II: Oboe 0
P2: Clarinetto I, II in Do/C: C Clarinet 0
P3: Fagotto I, II: Bassoon 0
P4: Clarino I, II in Do/C: C Clarinet 0
P5: Timpani in Do-Sol/C-G: Timpani 0
P6: Violino I: Violins 0
P7: Violino II: Violins 0
P8: Viola I, II: Violas 0
P9: Violoncello e Basso: Violoncellos 0
Creating df/cosi/07.xml
P1: Piano: Piano 0
P1: Piano: Piano 0
Creating df/merged_cosi/6.pkl
Creating df/cosi/00correct.musicxml
P1: Oboes I, II: Oboe 0
P2: Clarinetto I, II in Do/C: C Clarinet 0
P3: Fagotto I, II: Bassoon 0
P4: Clarino I, II in Do/C: C Cla

In [ ]:
#data = ScoreData(pr.load_df('input_with_chords', 10, use_cache=False))
#data.save('score_data/3.pkl')
data = ScoreData(pr.load_df('fun', 0))
new_data = data.copy()
for i in reductions:
    if 'Attempt 2' in i:
        new_data = reductions[i].predict(new_data, threshold=0.1, name='y_pred_%s' % i)
    else:
        new_data = reductions[i].predict_2(new_data, threshold=0.1, name='y_pred_%s' % i)

In [25]:
#data = ScoreData(pr.load_merged_df(9, use_cache=False))
#print(data.df.columns)
#data.save('score_data/9.pkl')

data = ScoreData.load('score_data/2.pkl')
#data = ScoreData(pr.load_df('fun', 0))
new_data = data.copy()

for i in reductions:
    if 'GRU_2' not in i or i[0] != '8':
        continue
    new_data = reductions[i].predict(new_data, threshold=0.3, name='y_pred_%s' % i)
    
output_names = []
count = 0
for i in reductions:
    if 'GRU_2' not in i or i[0] != '8':
        continue
    count += 1
    new_data.df['Reduction %s' % i.split(' ')[1]] = new_data.df['y_pred_%s' % i]
    output_names.append('Reduction %s' % i.split(' ')[1])
new_data.show_score(['x_train'] + output_names)

In [10]:
new_data.df['Comparison'] = new_data.get_color(new='y_pred_GRU_0 1', old='x_train')
new_data.df['Original'] = new_data.df['x_train']
new_data.df['Prediction'] = new_data.df['y_pred_GRU_0 1']
new_data.show_score(['Original', 'Prediction'], ['Comparison'])

In [8]:
print('%25s\t%20s\t%20s\t%20s' % ('', 'F1 score', 'Accuracy', 'Jaccard Similarity'))
reds = [i for i in sorted(list(reductions), key=lambda x:x[::-1]) if 'GRU' in i and '-' not in i]
for name in ['score_data/cosi_2.pkl', 'score_data/cosi_6.pkl', 'score_data/cosi_8.pkl']:
    new_data = check_data(name, reds, threshold=0.3)
    for i in list(reds) + ['LR using cosi_2', 'LR using cosi_6', 'LR using cosi_8', 
                           'NB using cosi_2', 'NB using cosi_6', 'NB using cosi_8', 'Keep All']:
        b1 = new_data.to_binary(col='y_train')
        b2 = new_data.to_binary(col='y_pred_%s' % i)
        scores = [metrics.f1_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  metrics.accuracy_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  pr.jaccard_similarity(b1, b2)
                 ]
        if 'GRU' in i:
            i = '%s using cosi_%c' % (i[2:], i[0])
        #elif 'Attempt 2 both ways' in i:
        #    i = '2 GRU using cosi_%c' % i[0]
        print('%25s\t%20.9f\t%20.9f\t%20.9f' % tuple([i] + scores))
    print()

                         	            F1 score	            Accuracy	  Jaccard Similarity
       GRU_1 using cosi_2	         0.840262582	         0.803234501	         0.724528302
       GRU_1 using cosi_6	         0.756862745	         0.665768194	         0.608832808
       GRU_1 using cosi_8	         0.710706150	         0.657681941	         0.551236749
       GRU_2 using cosi_2	         0.854771784	         0.811320755	         0.746376812
       GRU_2 using cosi_6	         0.783464567	         0.703504043	         0.644012945
       GRU_2 using cosi_8	         0.699551570	         0.638814016	         0.537931034
          LR using cosi_2	         0.714285714	         0.601078167	         0.555555556
          LR using cosi_6	         0.745454545	         0.622641509	         0.594202899
          LR using cosi_8	         0.545905707	         0.506738544	         0.375426621
          NB using cosi_2	         0.567639257	         0.560646900	         0.396296296
          NB using co

In [94]:
b2.shape

(171520,)

In [5]:
def check_data(name, reds, threshold=0.1):
    _, new_data = ScoreData.load(name).split_in(0.8, 0.9)
    
    features = ['active_rhythm', 'bass_line', 'entrance_effect', 'highest', 'in_chord', 'lowest', 'occurrence', 
       'onset_after_rest', 'rhythm_variety', 'strong_beats', 'sustained_rhythm', 'vertical_doubling', 'duration_length', 'offset']
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import MultinomialNB
    
    for i in [2, 6, 8]:
        training_data, _ = ScoreData.load('score_data/cosi_%d.pkl' % i).split_in(0.8, 0.9)
        training_data = training_data.df[training_data.df['x_train'] == 1].copy()

        x_train = training_data[features]
        y_train = training_data['y_train']
        model = LogisticRegression().fit(x_train, y_train)
        testing_data = new_data.df[new_data.df['x_train'] == 1].copy()
        testing_data['y_pred'] = model.predict(testing_data[features])
        new_data.df['y_pred_LR using cosi_%d' % i] = testing_data['y_pred']
        new_data.df['y_pred_LR using cosi_%d' % i] = new_data.df['y_pred_LR using cosi_%d' % i].fillna(0).astype(int)


        x_train = training_data[features]
        y_train = training_data['y_train']
        model = MultinomialNB().fit(x_train, y_train)
        testing_data = new_data.df[new_data.df['x_train'] == 1].copy()
        testing_data['y_pred'] = model.predict(testing_data[features])
        new_data.df['y_pred_NB using cosi_%d' % i] = testing_data['y_pred']
        new_data.df['y_pred_NB using cosi_%d' % i] = new_data.df['y_pred_NB using cosi_%d' % i].fillna(0).astype(int)

    new_data.df['y_pred_Keep All'] = new_data.df['x_train']
    new_data.df.columns
    
    for i in reds:
        if 'GRU' in i:
            new_data = reductions[i].predict(new_data, threshold=threshold, name='y_pred_%s' % i)
        else:
            new_data = reductions[i].predict_2(new_data, threshold=threshold, name='y_pred_%s' % i)
    return new_data

In [28]:
data = ScoreData.load('score_data/2.pkl')

In [11]:
import music21
name = 'GRU_0 with features 1'
x_test, y_test = reductions[name].generate_data(data,**reductions[name].params)
predictions = reductions[name].model.predict(x_test)
for i in predictions:
    results = []
    for j in range(i.shape[0]):
        results.append([music21.pitch.Pitch(j).nameWithOctave, i[j]])
    results = sorted(results, key=lambda x: -x[1])[:9]
    results = list(filter(lambda x: x[1] > 0.05 or x[1] == results[0][1], results))
    s = ' '.join(list(map(lambda x:'%3s(%f)' % (x[0], x[1]), results)))
    print(s)

 G2(0.999433)  G4(0.995289)  G3(0.876024)
 G4(0.999619)  G2(0.996449)  G3(0.349540)
 G2(0.999485)  G4(0.995229)
E-4(0.981923) E-2(0.978148)
 F2(0.993168)  F4(0.964552)  F3(0.913623)
 F4(0.999404)  F2(0.994467)
 F2(0.996440)  F4(0.911061)
 D2(0.993877)  D4(0.983297)  D3(0.090659)
 D4(0.997405)  D2(0.995198)
 G4(0.999913)
 G4(0.998297)
 G4(0.985198)
 C4(0.992610) E-4(0.987315)
G#4(0.991928)
G#4(0.999941)
G#4(0.999964)  F4(0.065459)
E-4(0.990481)  C4(0.885941)  G4(0.711153)
E-5(0.999808)
E-5(0.998158)
E-5(0.999811)
 C5(0.987456) E-4(0.986221)  G4(0.964766)  C4(0.962318)
 G4(0.994684)  C5(0.986845) E-4(0.974777)  C4(0.936459)
 G4(0.999895) E-4(0.077298)
 G4(0.998980)
 G4(0.991600)
 B3(0.996059)  D4(0.714277)
G#4(0.987965)
G#4(0.971621)
G#4(0.994488)
 B3(0.681111)  G4(0.470456)  D4(0.078540)
 F5(0.902523)  F4(0.442042)
 F5(0.998254)
 F5(0.987819)
 B3(0.986564)  D5(0.960177)  D4(0.761581)  G3(0.115314)  G4(0.081489)
 B3(0.987765)  D5(0.977772)  D4(0.705887)  B4(0.599558)  G4(0.334156)
 G5(0.

In [8]:

data = ScoreData.load('score_data/cosi_8.pkl')
data.show_score(['x_train', 'y_train'])

In [ ]:
new_data.df['Comparison'] = new_data.get_color(new='y_pred_GRU_0 1', old='x_train')
new_data.df['Original'] = new_data.df['x_train']
new_data.df['Prediction'] = new_data.df['y_pred_GRU_0 1']
new_data.show_score(['Original', 'Prediction'], ['Comparison'])